In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, Input

In [2]:
data= pd.read_csv(r"C:\Users\hsynt\Downloads\Selcuk Abi\Selcuk Abi\VeriSeti2.csv")

In [3]:
# Özellik ve Hedef Ayrımı
X = data.drop(columns=['Target_10', 'adjclose'])
y = data['Target_10']

# Zaman Serisi Formatına Getirme
def create_sequences(data, target, seq_length):
    sequences, targets = [], []
    for i in range(len(data) - seq_length):
        sequences.append(data[i:i + seq_length])
        targets.append(target[i + seq_length])
    return np.array(sequences), np.array(targets)

SEQ_LENGTH = 10
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_seq, y_seq = create_sequences(X_scaled, y.values, SEQ_LENGTH)

# PCA ve LDA Veri Hazırlığı
X_flat = X_seq.reshape(X_seq.shape[0], -1)  # Düzleştirilmiş veri

pca = PCA(n_components=10)
X_pca = pca.fit_transform(X_flat)

lda = LDA(n_components=1)
X_lda = lda.fit_transform(X_flat, y_seq)

# Performans Metriklerini Hesaplama
def calculate_metrics(y_true, y_pred, y_pred_proba):
    return {
        "Accuracy": accuracy_score(y_true, y_pred) * 100,
        "Precision": precision_score(y_true, y_pred) * 100,
        "Recall": recall_score(y_true, y_pred) * 100,
        "F1-Score": f1_score(y_true, y_pred) * 100,
        "AUC": roc_auc_score(y_true, y_pred_proba) * 100
    }

# CNN Modeli Tanımlama
def build_cnn_model(input_shape):
    model = Sequential([
        Input(shape=input_shape),
        Conv1D(64, kernel_size=2, activation='relu'),
        MaxPooling1D(pool_size=2),
        Flatten(),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Eğitim ve Test Performansını Hesaplama
results_train = []
results_test = []

for reduction, X_data in zip(
    ['No PCA/LDA', 'PCA', 'LDA'],
    [X_seq, X_pca, X_lda]
):
    if reduction == 'No PCA/LDA':
        X_data = X_data.reshape(X_data.shape[0], SEQ_LENGTH, -1)
    else:
        X_data = np.tile(X_data, (1, SEQ_LENGTH)).reshape(X_data.shape[0], SEQ_LENGTH, -1)

    # Eğitim ve Test Bölünmesi
    X_train, X_test, y_train, y_test = train_test_split(X_data, y_seq, test_size=0.2, random_state=42, stratify=y_seq)

    # Modeli Eğit
    cnn_model = build_cnn_model(X_train.shape[1:])
    cnn_model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)

    # Eğitim Kümesi Performansı
    y_train_pred = (cnn_model.predict(X_train) > 0.5).astype("int32").flatten()
    y_train_proba = cnn_model.predict(X_train).flatten()
    train_metrics = calculate_metrics(y_train, y_train_pred, y_train_proba)
    train_metrics['Reduction'] = reduction
    train_metrics['Model'] = 'CNN'
    results_train.append(train_metrics)

    # Test Kümesi Performansı
    y_test_pred = (cnn_model.predict(X_test) > 0.5).astype("int32").flatten()
    y_test_proba = cnn_model.predict(X_test).flatten()
    test_metrics = calculate_metrics(y_test, y_test_pred, y_test_proba)
    test_metrics['Reduction'] = reduction
    test_metrics['Model'] = 'CNN'
    results_test.append(test_metrics)

# Sonuçları Görselleştirme
train_df = pd.DataFrame(results_train)
test_df = pd.DataFrame(results_test)

# Eğitim ve Test Performans Sonuçları Tablosu
print("Eğitim Kümesi Performans Sonuçları:")
print(train_df)
print("\nTest Kümesi Performans Sonuçları:")
print(test_df)

# Sonuçları Kaydetme
train_df.to_csv("cnn_train_performance.csv", index=False)
test_df.to_csv("cnn_test_performance.csv", index=False)
print("\nSonuçlar 'cnn_train_performance.csv' ve 'cnn_test_performance.csv' dosyalarına kaydedildi.")

Epoch 1/10
1249/1249 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8247 - loss: 0.4058
Epoch 2/10
1249/1249 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8606 - loss: 0.3343
Epoch 3/10
1249/1249 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8668 - loss: 0.3207
Epoch 4/10
1249/1249 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8679 - loss: 0.3162
Epoch 5/10
1249/1249 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8694 - loss: 0.3095
Epoch 6/10
1249/1249 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8691 - loss: 0.3067
Epoch 7/10
1249/1249 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8741 - loss: 0.2996
Epoch 8/10
1249/1249 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8734 - loss: 0.2999
Epoch 9/10
1249/1249 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8737 - loss: 0.2996
Epoch 10/10
1249/1249 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8722 - loss: 0.3042
1249/1249 ━━━━━━━━━━━━━━━━━━━━ 1s 792us/step
1249/1249 ━━━━━━━━━━━━━━━━━━━━ 1s 762us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 